# Central Pattern Generator for control of quadruped robot

This is part 1 of the Legged robot course's project \#2. We will focus on implementing a montion controller based on a Central Pattern Generato (CPG).

# Importing the required libraries

In [1]:
#In the following we import the classes that we need for simulation
import time
import pybullet #pybullet simulator
import pybullet_data
import numpy as np # numpy library for matrix computatios

# import env.quadruped_motor as quadruped_motor
from env.quadruped import Quadruped # Quadruped class
import env.configs_a1 as robot_config 

%load_ext autoreload
%autoreload 2

In [2]:
#In the following we create an object of dynamic engine of pybullet and we connect it to the Pybullet GUI
# pybullet.disconnect()
pybullet_client = pybullet.connect(pybullet.GUI)

# pybullet.setAdditionalSearchPath(pybullet_data.getDataPath()) # TODO: Not needed ?

In [3]:
# Here we load the robot in the default position
robot = Quadruped(pybullet_client=pybullet,
                   robot_config=robot_config,
                   self_collision_enabled=True,
                   accurate_motor_model_enabled=True,
                   motor_control_mode="PD",
                   on_rack=False,
                   render=False)

In [ ]:
# time.sleep(2)
pybullet.disconnect()